## 데이터 클렌징

In [87]:
import pandas as pd
import re
from PyKomoran import *

komoran = Komoran("EXP") # komoran full version 모델 적용
komoran.set_user_dic("./user_dic/dic1.user") # 사용자 사전 등록

In [88]:
# 데이터 가져오기
dataframe = pd.read_excel("./output/03_data_output.xlsx")

In [101]:
# 전체 데이터에서 가장 많이 사용된 명사 단어 추출

# 텍스트 전처리
def clean_text(text):
    text = re.sub(r'\d+?\.\s\d+\.\s\d+\.', '', text) # 날짜 제거
    text = re.sub(r'\b\w+법원|지원', '', text) # 법원명 제거
    text = re.sub('\n[0-9]. ', '', text) # 목차 숫자 제거 1
    text = re.sub('\n\([0-9]\) ', '', text) # 목차 숫자 제거 2
    text = re.sub('\n[가-하]. ', '', text) # 목차 한글 제거
    text = re.sub('도로교통법위반|특정범죄가중처벌등에관한법률위반|수사보고|범 죄 사 실|범죄사실', '', text) # 위반 텍스트 제거
    return text

word_counter = {}
for text in dataframe['judgement_reason']:
    nouns = komoran.get_nouns(clean_text(text)) # 명사 추출
    nouns = list(filter(lambda x : len(x) > 1, nouns)) # 단어 길이가 2 이상인 단어만 추출
    # 워드 카운터
    for noun in nouns:
        if noun not in word_counter.keys():
            word_counter[noun] = 1
        else:
            word_counter[noun] += 1

print(sorted(word_counter.items(), key=lambda x:x[1], reverse=True)[:300])

[('피고인', 97917), ('운전', 38618), ('음주운전', 34030), ('형법', 32611), ('범행', 27101), ('차량', 25348), ('피해자', 24043), ('승용차', 21369), ('양형', 20810), ('상태', 20719), ('사건', 19946), ('도로', 19326), ('선택', 18043), ('정상', 16907), ('조의', 16748), ('정황', 14082), ('도로교통법', 13699), ('혈중알코올농도', 13646), ('서울', 13417), ('처벌', 13369), ('번호', 12944), ('생략', 12831), ('주취운전자', 12464), ('증거', 12292), ('이유', 11994), ('적용', 11212), ('해당', 11208), ('전력', 10957), ('요지', 10944), ('법정진술', 10882), ('징역형', 10718), ('집행유예', 10603), ('법조', 10521), ('법령', 10478), ('교통사고', 10066), ('자동차', 9866), ('사고', 9671), ('참작', 9146), ('주문', 9036), ('구간', 8745), ('결과', 8471), ('진행', 8380), ('상해', 8109), ('사람', 8083), ('범죄', 7995), ('성행', 7828), ('벌금형', 7748), ('정황진술보고서', 7705), ('위반', 7654), ('약식명령', 7572), ('종합', 7342), ('조건', 6968), ('부분', 6948), ('인정', 6883), ('종사', 6653), ('벌금', 6547), ('차로', 6476), ('명령', 6446), ('필요', 6347), ('환경', 6346), ('전방', 5997), ('범죄전력', 5888), ('조회', 5850), ('동기', 5638), ('변론', 5635), ('발생', 5631), ('치료',

In [84]:
text = dataframe['judgement_reason'][54]





print(text_nouns)

{'상태', '양형', '무면허운전', '범죄경력조회', '범행', '주문', '전력', '보고서', '기록', '정상', '조건', '참작', '적발', '정정', '고지', '시간', '증거', '안양', '이유', '도주차량', '집행유예', '음주운전', '식당', '벌금형', '동기', '형법', '나이', '부근', '요지', '지능', '음주운전단속결과통보', '결과', '법령', '쇼핑센터', '법정진술', '도로교통법', '생략', '선택', '조의', '서울', '운행', '구간', '승용차', '영등포구', '신길로', '피의자', '구로구', '적용', '도로', '번호', '사거리', '피고인', '약식명령', '이상', '차량', '벌금', '위반', '최초', '해당', '작량감경', '성행', '환경', '수단', '정황', '운전', '혈중알코올농도', '초과', '법조', '징역형'}


In [83]:
print(text)


피고인은   안양에서 (음주운전)죄로 벌금 500만 원의 약식명령을 고지받는 등 도로교통법 제44조 제1항을 위반한 전력이 있다.
피고인은  02:20분경 혈중알코올농도 0.134%의 술에 취한 상태에서 (차량번호 1 생략) 그랜져 승용차를 운전하여 서울 구로구 B역 부근 ‘C’ 식당 앞 도로에서 서울 영등포구 신길로 145 ‘사러가 쇼핑센터’ 사거리 도로까지 약 3km 구간을 운행하였다. 이로써 피고인은 도로교통법 제44조 제1항을 2회 이상 위반하였다.
증거의 요지피고인의 법정진술적발보고서, 음주운전단속결과통보, 서(피의자 최초 적발시간 정정 등)범죄경력조회
법령의 적용에 대한 해당법조 및 형의 선택
도로교통법 제148조의2 제3항 제1호, 제44조 제1항. 징역형 선택작량감경
형법 제53조, 제55조 제1항 제3호집행유예
형법 제62조 제1항
양형의 이유
○ 유리한 정상: 벌금형을 초과하는 전과 없음
○ 불리한 정상: 2007, 2008년 음주운전으로, 2012년 (도주차량)죄 등으로, 2018년 무면허운전으로 각각 벌금형을 받은 전력이 있음
○ 위와 같은 정상과 피고인의 나이, 성행, 지능과 환경, 범행의 동기, 수단과 결과, 범행 후의 정황 등 기록에 나타난 양형 조건을 참작하여 주문과 같이 형을 정한다.
